In [1]:
!pip install requests Pillow


In [4]:
import requests
from PIL import Image
from io import BytesIO
import time

def find_jpeg(data):
    """
    Find JPEG images within a stream of data.
    Yields start and end indices of JPEG images.
    """
    start = 0
    while True:
        start = data.find(b'\xff\xd8', start)
        if start == -1:
            break
        end = data.find(b'\xff\xd9', start) + 2
        if end == 1:
            break
        yield start, end
        start = end

def save_frames_from_stream(url, timeout=30, max_frames=1):
    """
    Connects to the ESP32 camera stream, captures and saves frames as JPEG files.

    Args:
    - url: The URL of the camera stream.
    - timeout: How long to wait for a response, in seconds.
    - max_frames: The maximum number of frames to save.
    """
    response = requests.get(url, stream=True, timeout=timeout)
    if response.status_code == 200:
        bytes_io = BytesIO()
        frame_count = 0
        for chunk in response.iter_content(chunk_size=1024):
            bytes_io.write(chunk)
            # Search for JPEG images in the accumulated data
            data = bytes_io.getvalue()
            for start, end in find_jpeg(data):
                # Extract and save the JPEG image
                img_data = data[start:end]
                img = Image.open(BytesIO(img_data))
                filename = f'frame_{int(time.time())}_{frame_count}.jpg'
                img.save(filename)
                print(f'Saved {filename}')
                frame_count += 1
                if frame_count >= max_frames:
                    return
            # Clear the buffer if it's getting too large
            if bytes_io.tell() > 10 * 1024 * 1024:  # 10MB
                bytes_io.seek(0)
                bytes_io.truncate()
    else:
        print("Failed to connect to the camera stream.")

if __name__ == "__main__":
    # Your ESP32 camera stream URL
    STREAM_URL = "http://192.168.137.149"
    save_frames_from_stream(STREAM_URL, max_frames=1)


KeyboardInterrupt: 

: 